# Training 

The experiments are logged using Wandb 

In [ ]:
!pip install -U git+https://github.com/aguirrejuan/ConvRFF.git --quiet

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.6 MB/s eta 0:00:00


In [ ]:
import wandb 

sweep_config = {
    'method':'grid'
}

parameters_dict = {
    'model': {
        'values':['f_b','r_b','u_b','f_b_s_m3','r_b_s_m3','u_b_s_m3', 'f_b_s', 'f_r_s', 
                   'f_r_s_m1','r_b_s','r_r_s','r_r_s_m1', 'u_r_s_m1','u_r_s','u_b_s','u_r_b']
        },
    'dataset':{
        'values':['infrared_thermal_feet_nodataaug', 'infrared_thermal_feet']
    }
    }
sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project='thesis_experiments')

Create sweep with ID: hg11kida
Sweep URL: https://wandb.ai/colab11/thesis_experiments/sweeps/hg11kida


In [ ]:
from tensorflow.keras import regularizers


from functools import partial 
from convRFF import train as train_model
from gcpds.image_segmentation.datasets.segmentation import NerveUtp
from gcpds.image_segmentation.datasets.segmentation import BrachialPlexus
from gcpds.image_segmentation.datasets.segmentation import OxfordIiitPet
from gcpds.image_segmentation.datasets.segmentation import InfraredThermalFeet
import wandb

from gcpds.image_segmentation.models import (fcn_baseline,
                                             unet_baseline,
                                             res_unet_baseline
)
from convRFF.models.unet import unet_b_skips, unet_rff_skips, unet_rff_backbone
from convRFF.models.fcn import fcn_b_skips, fcn_rff_skips, fcn_rff_backbone
from convRFF.models.res_unet import res_unet_b_skips, res_unet_rff_skips, res_unet_rff_backbone


#===============================================================================
import tensorflow as tf 
from gcpds.image_segmentation.losses import DiceCoefficient
from gcpds.image_segmentation.metrics import (Jaccard, 
                                              Sensitivity,
                                              Specificity,
                                              DiceCoefficientMetric)



def compile_parameters():
  return {'loss':DiceCoefficient(),
          'optimizer':tf.keras.optimizers.Adam(learning_rate=1e-3),
          'metrics':[Jaccard(),
                     Jaccard(name='Jaccard_0',target_class=0),
                     Jaccard(name='Jaccard_1',target_class=1),
                     Jaccard(name='Jaccard_2',target_class=2),
                     Sensitivity(),
                     Sensitivity(name='Sensitivity_0',target_class=0),
                     Sensitivity(name='Sensitivity_1',target_class=1),
                     Sensitivity(name='Sensitivity_2',target_class=2),
                     Specificity(),
                     Specificity(name='Specificity_0',target_class=0),
                     Specificity(name='Specificity_1',target_class=1),
                     Specificity(name='Specificity_2',target_class=2),
                     DiceCoefficientMetric(),
                     DiceCoefficientMetric(name='DiceCoeficienteMetric_0',target_class=0),
                     DiceCoefficientMetric(name='DiceCoeficienteMetric_1',target_class=1),
                     DiceCoefficientMetric(name='DiceCoeficienteMetric_2',target_class=2),
                     ]
  }




def get_model(model, input_shape, out_channels, kernel_regularizer):
    MODELS = {'u_b':unet_baseline(input_shape=input_shape, out_channels=out_channels),
              'u_b_s':unet_b_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer),
              'u_r_s':unet_rff_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer, mul_dim=3),
              'u_r_s_m1':unet_rff_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer, mul_dim=1),
              'u_b_s_m3':unet_b_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer, mul_dim=3) ,
              'u_r_b':unet_rff_backbone(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer, mul_dim=3),
              'f_b':fcn_baseline(input_shape=input_shape,out_channels=out_channels),
              'r_b':res_unet_baseline(input_shape=input_shape,out_channels=out_channels),
              'f_b_s':fcn_b_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer) ,
              'f_r_s':fcn_rff_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer) ,
              'f_r_s_m1':fcn_rff_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer,mul_dim=1),
              'f_b_s_m3':fcn_b_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer,mul_dim=3),
              'r_b_s':res_unet_b_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer),
              'r_b_s_m3':res_unet_b_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer,mul_dim=3),
              'r_r_s':res_unet_rff_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer), 
              'r_r_s_m1':res_unet_rff_skips(input_shape=input_shape,out_channels=out_channels,kernel_regularizer=kernel_regularizer,mul_dim=1)
              }
    return MODELS[model]

In [ ]:
from convRFF.train import get_compile_parameters as gcp
from convRFF.train import get_train_parameters as gtp


def get_t_p(*args,**kwargs):
    f = gtp(*args,**kwargs)
    f.update({'epochs':200})
    return f

def get_c_p(*args,**kwargs):
    f = gcp(*args,**kwargs)
    f.update({'optimizer':tf.keras.optimizers.Adam(learning_rate=1e-3)})
    return f

kwargs_data_augmentation = dict(repeat=7, 
                                   flip_left_right=True, 
                                   flip_up_down=False,
                                   range_rotate=(-15,15),
                                   translation_h_w=(0.10,0.10),
                                   zoom_h_w=(0.15,0.15),
                                   batch_size=16,
                                   shape=224,
                                    split=[0.1,0.1]
                                   )
kernel_regularizer =None

get_compile_parameters = get_c_p
get_train_parameters = get_t_p
out_channels = 1
input_shape = 224,224,1

In [ ]:
def train(config=None):
    with wandb.init(config=config) as run:
        config= dict(run.config)
        model = config['model']
        name_dataset = config['dataset']

        if name_dataset== 'infrared_thermal_feet': 
            data_augmentation = True
        elif name_dataset== 'infrared_thermal_feet_nodataaug':
            data_augmentation = False
            kwargs_data_augmentation['repeat'] = 1

        dataset_class = InfraredThermalFeet
        model = get_model(model, input_shape, out_channels, kernel_regularizer)
        train_model(model, dataset_class, run,  data_augmentation=data_augmentation, 
                                                get_compile_parameters=get_compile_parameters,
                                                get_train_parameters=get_train_parameters,
                                                **kwargs_data_augmentation)
import os 
os.environ["WANDB_API_KEY"] = "YOUR_WANDB_API_KEY"
SWEEP_ID = 'YOUR_SWEEP_ID'
wandb.agent(SWEEP_ID, train, count=1000, project='thesis_experiments')